In [1]:
import pandas as pd
import numpy as np

In [2]:
outputs_2010 = pd.read_csv('EPA_CMAQ/outputs/2010_pm25_daily_average 2.txt') 

In [3]:
outputs_2010['s_FIPS'] = outputs_2010['FIPS'].astype(str) #= outputs_2010.apply(lambda x: str(x.FIPS)[0]=='6', axis=1)

In [4]:
ca_outputs_2010 = outputs_2010[outputs_2010.s_FIPS.str[0] =='6']

In [5]:
ca_outputs_2010.head()

,Date,FIPS,Longitude,Latitude,pm25_daily_average(ug/m3),pm25_daily_average_stderr(ug/m3),s_FIPS
3391,2010/01/01,6001400100,-122.23181,37.86754,24.762,7.2211,6001400100
3392,2010/01/01,6001400200,-122.24948,37.84817,24.861,7.4563,6001400200
3393,2010/01/01,6001400300,-122.25442,37.84056,24.545,7.2497,6001400300
3394,2010/01/01,6001400400,-122.25752,37.84801,24.904,7.4018,6001400400
3395,2010/01/01,6001400500,-122.26480,37.84853,24.300,7.2837,6001400500


In [6]:
ca_annual_2010 = ca_outputs_2010[['Longitude', 'Latitude', 'FIPS', 'pm25_daily_average(ug/m3)']].\
groupby('FIPS').mean().reset_index()

In [7]:
# round to nearest 0.05 
ca_annual_2010['approx_Long'] = round(ca_annual_2010.Longitude/0.005) * 0.005
ca_annual_2010['approx_Lat'] = round(ca_annual_2010.Latitude/0.005) * 0.005

In [8]:
ref = pd.read_csv('/Users/linghe/Desktop/Columbia/2019 Fall/Capstone/calibre/example/data/dh_ca_annual_2010.csv') 

In [9]:
ref['approx_Long'] = round(ref.x/0.005) * 0.005
ref['approx_Lat'] = round(ref.y/0.005) * 0.005

In [10]:
ref.head()

,x,y,PM25,approx_Long,approx_Lat
0,-124.405,42.005,NaN,-124.405,42.005
1,-124.395,42.005,NaN,-124.395,42.005
2,-124.385,42.005,NaN,-124.385,42.005
3,-124.375,42.005,NaN,-124.375,42.005
4,-124.365,42.005,NaN,-124.365,42.005


In [11]:
ca_annual_2010.head()

,FIPS,Longitude,Latitude,pm25_daily_average(ug/m3),approx_Long,approx_Lat
0,6001400100,-122.23181,37.86754,9.780627,-122.230,37.87
1,6001400200,-122.24948,37.84817,9.902219,-122.250,37.85
2,6001400300,-122.25442,37.84056,9.863416,-122.255,37.84
3,6001400400,-122.25752,37.84801,9.854425,-122.260,37.85
4,6001400500,-122.26480,37.84853,9.832364,-122.265,37.85


In [12]:
ca_annual_2010.drop_duplicates(subset = ['approx_Long','approx_Lat'], 
                     keep = False, inplace = True) 

In [13]:
joined_df = ref.merge(ca_annual_2010[['Longitude', 'Latitude', 'pm25_daily_average(ug/m3)', 'approx_Long', 'approx_Lat']], \
         how='left', on=['approx_Long','approx_Lat'])

In [14]:
joined_df.shape

(974544, 8)

In [15]:
cmaq_ca_annual_2010 = joined_df[['x', 'y', 'pm25_daily_average(ug/m3)']]

In [16]:
cmaq_ca_annual_2010.rename(columns={'x':'lon',
                          'y':'lat',
                          'pm25_daily_average(ug/m3)': 'pm25'}, 
                 inplace=True)

/anaconda3/envs/dlenv/lib/python3.6/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [17]:
cmaq_ca_annual_2010.shape

(974544, 3)

In [18]:
# cmaq_ca_annual_2010.to_csv('/Users/linghe/Desktop/Columbia/2019 Fall/Capstone/'+\
# 'calibre/example/data/CMAQ_2010_align.csv')

In [19]:
cmaq_ca_annual_2010_1 = cmaq_ca_annual_2010.copy()
cmaq_ca_annual_2010_2 = cmaq_ca_annual_2010.copy()

fake_pm25 = np.random.uniform(ca_annual_2010['pm25_daily_average(ug/m3)'].min(), \
                              ca_annual_2010['pm25_daily_average(ug/m3)'].max(), (2, ref.shape[0]))

In [20]:
cmaq_ca_annual_2010_1['pm25'] = fake_pm25[0]
cmaq_ca_annual_2010_2['pm25'] = fake_pm25[1]

In [21]:
cmaq_ca_annual_2010_1.to_csv('/Users/linghe/Desktop/Columbia/2019 Fall/Capstone/'+\
'calibre/example/data/CMAQ1_2010_align.csv')

In [22]:
cmaq_ca_annual_2010_2.to_csv('/Users/linghe/Desktop/Columbia/2019 Fall/Capstone/'+\
'calibre/example/data/CMAQ2_2010_align.csv')